In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
df = pd.read_csv('fashion.csv')
df.head()

,date,Cami Dresses,Shirts,Tote Bags,Sneakers,Crop Tops,Polos,Cross Body Bags,Casual Jackets,Swimwear,...,Heels,T-Shirts,Activewear Tops & T-Shirts,Watches & Timepieces,Wallets & Card Holders,Bodycon Dresses,Beauty Tools & Accessories,Skinny Jeans,Beauty Eyes,Beauty Face
0,2017-08-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-08-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-08-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-08-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()
df = df.iloc[:,1:]
df_copy = df.copy()

In [4]:
num_layers = 1
learning_rate = 0.01
size_layer = 128
timestamp = 5
epoch = 500
dropout_rate = 0.7
future_weeks = 30

In [5]:
class Model(tf.keras.Model):
    def __init__(self, learning_rate, num_layers, size, size_layer, forget_bias=0.8):
        super(Model, self).__init__()
        self.num_layers = num_layers
        self.size_layer = size_layer
        self.learning_rate = learning_rate
        
        # Define LSTM layers with dropout
        self.lstm_layers = [
            tf.keras.layers.LSTM(
                size_layer,
                return_sequences=True if i < num_layers - 1 else False,
                return_state=True
            )
            for i in range(num_layers)
        ]
        self.dropout = tf.keras.layers.Dropout(1 - forget_bias)
        
        # Output dense layer
        self.dense = tf.keras.layers.Dense(size, kernel_initializer=tf.keras.initializers.GlorotUniform())

        # Optimizer
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)

    def call(self, inputs, hidden_states):
        x = inputs
        states = []
        
        # Pass through each LSTM layer
        for i in range(self.num_layers):
            x, h, c = self.lstm_layers[i](x, initial_state=hidden_states[i])
            states.append([h, c])
        
        # Apply dropout to the final output
        x = self.dropout(x)
        
        # Output dense layer
        logits = self.dense(x)
        return logits, states

    def compute_loss(self, logits, labels):
        return tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        )
    
    def train_step(self, inputs, labels, hidden_states):
        with tf.GradientTape() as tape:
            logits, states = self.call(inputs, hidden_states)
            loss = self.compute_loss(logits, labels)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss, states

In [7]:
# Instantiate the model
modelnn = Model(learning_rate, num_layers, df.shape[1], size_layer, dropout_rate)

# Example: Directly run model operations
# Pass inputs and perform operations to initialize variables
example_input = tf.random.normal((1, 10, df.shape[1]))  # Example shape (batch_size, sequence_length, feature_size)
example_hidden_state = [
    [tf.zeros((1, size_layer)), tf.zeros((1, size_layer))]  # Initial hidden state
    for _ in range(num_layers)
]

# Run a forward pass to initialize the model
outputs, updated_states = modelnn(example_input, example_hidden_state)

print("Model initialized successfully.")

Model initialized successfully.


In [9]:
for i in range(epoch):
    init_value = np.zeros((1, num_layers * 2 * size_layer))
    total_loss = 0
    for k in range(0, (df.shape[0] // timestamp) * timestamp, timestamp):
        batch_x = np.expand_dims(df.iloc[k: k + timestamp].values, axis = 0)
        batch_y = df.iloc[k + 1: k + timestamp + 1].values
        last_state, _, loss = sess.run([modelnn.last_state, 
                                        modelnn.optimizer, 
                                        modelnn.cost], feed_dict={modelnn.X: batch_x, 
                                                                  modelnn.Y: batch_y, 
                                                                  modelnn.hidden_layer: init_value})
        init_value = last_state
        total_loss += loss
    total_loss /= (df.shape[0] // timestamp)
    if (i + 1) % 100 == 0:
        print('epoch:', i + 1, 'avg loss:', total_loss)

NameError: name 'sess' is not defined